# Formula One Insights with Pyton & SQL
Since its inception in the 1950s, Formula One has represented the pinnacle of global motorsport, pushing the boundaries of racing and automotive engineering. This analysis leverages Python and SQL to uncover insights into the achievements of drivers and constructors across F1's decades-long history.

*For a more detailed exploration, please refer to the accompanying PDF document. The comments within this Jupyter notebook are provided exclusively to explain the functionality of the code.*
*The values and data in this Jupyter Notebook were last updated on the 17th of January 2025.*

# Organizing the Notebook into Multiple Parts
Due to the extensive code in this notebook, it has been divided into five parts. The first notebook focuses on retrieving data from the formula1.com website. The second notebook handles data retrieval from the F1DB database. The third notebook is dedicated to creating statistics and visualizations. The fourth notebook explores the question of who is the Greatest Driver of All Time. The fifth notebook consolidates multiple CSV files into separate Excel worksheets.

# The Greatest Driver of All Time
The phrase "greatest of all time" is often heard in the world of Formula One. However, it is challenging to definitively identify a single driver as the GOAT, given how much the sport has evolved over the years. Would someone like Hamilton or Verstappen have been as competitive in the front-engine machinery of the 1950s? Could Lauda or Senna have matched the fitness and athleticism required of today’s drivers?

While Lewis Hamilton is widely regarded as the greatest Formula One driver of all time, we will conduct a thought experiment. By considering a range of factors, we will develop a rating for each of the 12 Formula One eras. This rating will then be used as a multiplier to adjust and scale the achievements of each driver. Finally, we will analyze the results to see how they compare.

*For more details, please refer to the final section of the accompanying PDF file.*

# Importing Python Libraries
This Jupyter notebook is designed to run on most modern Python installations. However, to ensure reproducibility, note that it was developed and tested with Python 3.12.3. The following libraries and their respective versions were used in this analysis:

- pandas 2.2.2
- scikit-learn 1.5.0

In [1]:
# Import libraries
import os
import sqlite3
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

print("Libraries imported")

Libraries imported


# Creating the GreatestOfAllTime Table
We consolidate all the drivers from the previous top 10 classifications into a single table.

In [2]:
# Connect to the SQLite database
conn = sqlite3.connect('formula one.db')
cursor = conn.cursor()

# Create the GreatestOfAllTime table
create_table_query = '''
CREATE TABLE IF NOT EXISTS GreatestOfAllTime (
    Driver TEXT PRIMARY KEY,
    ChampionshipWins INTEGER,
    RaceWins INTEGER,
    PolePositions INTEGER,
    PodiumPositions INTEGER,
    FastestLaps INTEGER
);
'''
cursor.execute(create_table_query)

# List of drivers to be included
drivers = [
    "Michael Schumacher", "Lewis Hamilton", "Juan Manuel Fangio", "Sebastian Vettel", "Max Verstappen", 
    "Alain Prost", "Niki Lauda", "Nelson Piquet", "Jackie Stewart", "Jack Brabham", "Ayrton Senna", 
    "Fernando Alonso", "Nigel Mansell", "Jim Clark", "Nico Rosberg", "Kimi Raikkonen", 
    "Rubens Barrichello", "Valtteri Bottas", "Mika Hakkinen"
]

# Insert drivers into the table with default values
for driver in drivers:
    insert_query = '''
    INSERT OR IGNORE INTO GreatestOfAllTime (Driver, ChampionshipWins, RaceWins, PolePositions, PodiumPositions, FastestLaps)
    VALUES (?, 0, 0, 0, 0, 0);
    '''
    cursor.execute(insert_query, (driver,))

# Commit the transaction and close the connection
conn.commit()
conn.close()

print("GreatestOfAllTime table created and drivers inserted.")

GreatestOfAllTime table created and drivers inserted.


# Populating the GreatestOfAllTime Table with Driver Achievements
We populate the table columns with the drivers’ achievements, retrieving the data from other tables in the database. Additionally, we create a DriverRating column, which represents the sum of all achievements a driver has accomplished.

In [3]:
# Connect to the SQLite database
conn = sqlite3.connect('formula one.db')
cursor = conn.cursor()

# Check if the DriverRating column already exists
cursor.execute("""
    SELECT 1 FROM pragma_table_info('GreatestOfAllTime') WHERE name = 'DriverRating'
""")
column_exists = cursor.fetchone()

# Add the column only if it does not exist
if not column_exists:
    cursor.execute("ALTER TABLE GreatestOfAllTime ADD COLUMN DriverRating INTEGER")

# Populate ChampionshipWins
cursor.execute("""
    UPDATE GreatestOfAllTime
    SET ChampionshipWins = (
        SELECT COUNT(*)
        FROM DriverStandings
        WHERE DriverStandings.Driver = GreatestOfAllTime.Driver
          AND Position = 1
    )
""")

# Populate RaceWins
cursor.execute("""
    UPDATE GreatestOfAllTime
    SET RaceWins = (
        SELECT COUNT(*)
        FROM RaceWinners
        WHERE RaceWinners.Driver = GreatestOfAllTime.Driver
    )
""")

# Populate PolePositions
cursor.execute("""
    UPDATE GreatestOfAllTime
    SET PolePositions = (
        SELECT COUNT(*)
        FROM QualifyingResults
        WHERE QualifyingResults.Driver = GreatestOfAllTime.Driver
          AND Position = 1
    )
""")

# Populate PodiumPositions
cursor.execute("""
    UPDATE GreatestOfAllTime
    SET PodiumPositions = (
        SELECT COUNT(*)
        FROM RaceResults
        WHERE RaceResults.Driver = GreatestOfAllTime.Driver
          AND Position IN (1, 2, 3)
    )
""")

# Populate FastestLaps
cursor.execute("""
    UPDATE GreatestOfAllTime
    SET FastestLaps = (
        SELECT COUNT(*)
        FROM FastestLaps
        WHERE FastestLaps.Driver = GreatestOfAllTime.Driver
    )
""")

# Populate DriverRating by summing the relevant columns
cursor.execute("""
    UPDATE GreatestOfAllTime
    SET DriverRating = 
        ChampionshipWins + RaceWins + PolePositions + PodiumPositions + FastestLaps
""")

# Commit the changes and close the connection
conn.commit()
conn.close()

print("GreatestOfAllTime table has been updated with statistics and DriverRating.")

GreatestOfAllTime table has been updated with statistics and DriverRating.


# Calculating Ratings for each Formula One Era
We load the manually edited *era ratings.csv* file, normalize the values, and invert the scales for specific columns as needed.

In [4]:
# Load the era ratings file into a DataFrame
file_path = "era ratings.csv"
df = pd.read_csv(file_path)

# Select the numerical columns
columns_to_normalize = ['DriversPerEra', 'ConstructorsPerEra', 'RoundsPerEra', 'OverallSafety', 'SpeedAndManeuverability']

# Create a scaler object
scaler = MinMaxScaler()

# Normalize the columns
df_normalized = df.copy()
df_normalized[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])

# Invert scales for RoundsPerEra, OverallSafety, SpeedAndManeuverability
columns_to_invert = ['RoundsPerEra', 'OverallSafety', 'SpeedAndManeuverability']
df_normalized[columns_to_invert] = 1 - df_normalized[columns_to_invert]

# Calculate EraRating
df_normalized['EraRating'] = df_normalized[columns_to_normalize].sum(axis=1)

# Normalize EraRating
df_normalized['EraRatingNormalized'] = scaler.fit_transform(df_normalized[['EraRating']])

# Print the final ratings
print(df_normalized[['Era', 'EraRatingNormalized']].round(2))

                                                  Era  EraRatingNormalized
0              1950-1957: Front-Engine / Factory Cars                 1.00
1          1958-1961: Rear-Mid Engine / British Teams                 0.82
2        1962-1967: 1.5L Engines / Anglophone Drivers                 0.57
3   1968-1976: 12-Cylinder Engines / Sponsors / Sa...                 0.61
4                            1977-1982: Ground-Effect                 0.63
5                1983-1988: 1.5L Turbocharged Engines                 0.38
6         1989-1994: 3.5L Naturally Aspirated Engines                 0.53
7                               1995-1999: 3L Engines                 0.12
8                              2000-2005: V10 Engines                 0.23
9                          2006-2013: 2.4L V8 Engines                 0.25
10               2014-2021: 1.6L Turbocharged Engines                 0.01
11            2022-2025: Ground-Effect / Aero Changes                 0.00


# Scaling the Original Driver Ratings
We apply the era ratings calculated above to scale the achievements of drivers in the GreatestOfAllTime table. Since most drivers competed across multiple eras, we calculate average era ratings for each driver. Using a simple formula, we adjust the original values and display the newly scaled results.

In [5]:
# Connect to the SQLite database
conn = sqlite3.connect('formula one.db')

# Extract the GreatestOfAllTime table into a Pandas DataFrame
query = "SELECT * FROM GreatestOfAllTime"
goat_df = pd.read_sql_query(query, conn)

# Convert all numerical columns to float
numerical_columns = goat_df.select_dtypes(include=['int64', 'float64']).columns
goat_df[numerical_columns] = goat_df[numerical_columns].astype(float)

# Close the connection
conn.close()

# Define the multipliers for each driver

# For Senna and Clark, we apply a 10% increase to account for their untimely deaths, 
# as they likely would have raced longer and achieved more results
row_multipliers = {
    "Michael Schumacher": 0.2825,
    "Lewis Hamilton": 0.0866,
    "Juan Manuel Fangio": 0.9100,
    "Sebastian Vettel": 0.0866,
    "Max Verstappen": 0.0050,
    "Alain Prost": 0.5133,
    "Niki Lauda": 0.5400,
    "Nelson Piquet": 0.5133,
    "Jackie Stewart": 0.5900,
    "Jack Brabham": 0.7500,
    "Ayrton Senna": 0.4095, # Initial value 0.4550
    "Fernando Alonso": 0.1225,
    "Nigel Mansell": 0.4150,
    "Jim Clark": 0.5999, # Initial value 0.6666
    "Nico Rosberg": 0.1300,
    "Kimi Raikkonen": 0.1633,
    "Rubens Barrichello": 0.2825,
    "Valtteri Bottas": 0.0866,
    "Mika Hakkinen": 0.2933
}

# Apply the scaling formula to each row based on the driver
columns_to_scale = ["ChampionshipWins", "RaceWins", "PolePositions", "PodiumPositions", "FastestLaps", "DriverRating"]
for index, row in goat_df.iterrows():
    driver = row["Driver"]
    if driver in row_multipliers:
        multiplier = row_multipliers[driver]
        for column in columns_to_scale:
            scaled_value = round(row[column] + row[column] * multiplier, 2)
            goat_df.at[index, column] = scaled_value

# Display the scaled results
print(goat_df)

                Driver  ChampionshipWins  RaceWins  PolePositions  \
0   Michael Schumacher              8.98    116.71          87.21   
1       Lewis Hamilton              7.61    114.09         113.01   
2   Juan Manuel Fangio              9.55     45.84          55.39   
3     Sebastian Vettel              4.35     57.59          61.94   
4       Max Verstappen              4.02     63.31          40.20   
5          Alain Prost              6.05     77.18          49.94   
6           Niki Lauda              4.62     38.50          36.96   
7        Nelson Piquet              4.54     34.81          36.32   
8       Jackie Stewart              4.77     42.93          27.03   
9         Jack Brabham              5.25     24.50          22.75   
10        Ayrton Senna              4.23     57.79          91.62   
11     Fernando Alonso              2.25     35.92          24.70   
12       Nigel Mansell              1.42     43.87          45.28   
13           Jim Clark            

# Creating the GreatestOfAllTime Table
We add the drivers' scaled results to the database.

In [6]:
# Connect to the SQLite database
conn = sqlite3.connect('formula one.db')

# Save the DataFrame as a new table in the database
goat_df.to_sql('GreatestOfAllTimeScaled', conn, if_exists='replace', index=False)

# Close the connection
conn.close()

print("GreatestOfAllTimeScaled table has been successfully created in the database.")

GreatestOfAllTimeScaled table has been successfully created in the database.
